In [5]:
import os, sys
import numpy as np
import imageio
import pprint
import matplotlib.pyplot as plt

In [6]:

import os
import sys
import copy
import pickle

ROOT_DIR = os.getcwd()
sys.path.insert(0, os.path.join(ROOT_DIR, ".."))
sys.path.insert(0, os.path.join(ROOT_DIR, "src"))
sys.path.insert(0, os.path.join(ROOT_DIR, "inerf_utils"))

import json
import torch
import numpy as np
import torchvision.transforms as T
import imageio
import cv2
import matplotlib.pyplot as plt
import wisp
from wisp.core import RenderBuffer, Rays

from tqdm import tqdm
from scipy.spatial.transform import Rotation as R

from wisp.framework import WispState
from inerf_utils import *
from kaolin.render.camera import Camera, PinholeIntrinsics, CameraExtrinsics, CameraIntrinsics, blender_coords, opengl_coords
from kaolin.render.camera.extrinsics_backends import _Matrix6DofRotationRep

from wisp.ops.raygen import generate_pinhole_rays, generate_centered_pixel_coords
from wisp.datasets import SampleRays
from wisp.trainers import MultiviewTrainer
from wisp.models.pipeline import Pipeline

from scipy.optimize import minimize
from scipy.optimize import differential_evolution as de
from scipy.linalg import expm
from scipy.linalg import logm

import plotly.express as px

base_scale = 6
base_offset = torch.tensor([0, -0.6, -0.15])
translation_scale = torch.tensor(base_scale/1.25)
translation_offset = translation_scale * base_offset

# parser = argparse.ArgumentParser(description='iNeRF')
# parser.add_argument('--config', type=str, default='configs/nerf_hash.yaml', help='Path to config file.')
# parser.add_argument('--valid-only', action='store_true', help='Only validate.', default=True)

# parser.add_argument('-mn', '--model-name', type=str, default='', help='name of pretrained model.')
# parser.add_argument('-dp', '--dataset-path', type=str, default='', help='Path to dataset.')
# parser.add_argument('-inp', '--input', type=str, help='Path to input directory.')
# parser.add_argument('-out', '--output', type=str, help='Path to output directory.')
# parser.add_argument('--num-starts', type=int, default=5, help='Number of initial poses to start from in optimization')
# args = parser.parse_args()

In [7]:

# model_name = 'workdir/cheezit_5_2_val4_noflip_flip_21_new_2/rob0_cheezit_single_side_env_nobg_sam_iteration_29'
# root_dir = '/home/saptarshi/dev/kaolin-wisp/_results3/ensembles/' + model_name + '/'
# model_dir = os.path.join(root_dir, f"model_1")
# model_path = os.path.join(model_dir, list(sorted(os.listdir(model_dir)))[0], "model.pth")

model_path = '/home/saptarshi/dev/kaolin-wisp/_results3/ensembles/workdir/cheezit_5_2_val4_noflip_flip_21_new_2/rob0_cheezit_single_side_env_nobg_sam_iteration_29/model_1/20240202-211529/model.pth'

sys.argv[1:] = [
    '--config=../app/nerf/configs/nerf_hash.yaml',
    f'--pretrained={model_path}',
    f'--dataset-path=/home/saptarshi/dev/CustomComposer/workdir/cheezit_5_2_val4_noflip_flip_21_new_2/merged_images_0',
    '--valid-only'
]

W, H = 200, 200


fx = (0.5 * H) / np.tan(0.5 * float(1.3213687585295282))
extra_args = {
  'resume' : True,
  'output' : './pose_estimation',
  'device' : 'cuda',
  'z_near' : 0.0,
  'z_far' : 10.0,
  'focal' : fx,
  'lrate' : 1e-3,
}


def make_model(args, args_dict, extra_args, train_dataset, validation_dataset):
  pipeline = torch.load(args.pretrained)
  pipeline.to(extra_args['device'])
  scene_state = WispState()
  trainer = load_trainer(
      pipeline=pipeline,
      train_dataset=train_dataset, 
      validation_dataset=validation_dataset,
      device=extra_args['device'], 
      scene_state=scene_state,
      args=args, 
      args_dict=args_dict
  )
  return trainer, pipeline

def gen_camera(pose, focal, w, h, far):
  view_matrix = torch.linalg.inv(pose)

  x0 = 0.0
  y0 = 0.0

  camera = Camera.from_args(
      view_matrix=view_matrix,
      focal_x=focal,
      focal_y=focal,
      width=w,
      height=h,
      far=far,
      near=0.0,
      x0=x0,
      y0=y0,
      dtype=torch.float32,
      device=extra_args['device']
  )
  camera.change_coordinate_system(blender_coords())

  return camera

def gen_rays_only(camera):
  ray_grid = generate_centered_pixel_coords(camera.width, camera.height, camera.width, camera.height, device=extra_args['device'])
  rays = generate_pinhole_rays(camera, ray_grid).reshape(camera.height, camera.width, 3)
  rays = Rays.stack(rays)
  return rays

inerf_args, inerf_args_dict = parse_args()
train_dataset, validation_dataset = load_dataset(args=inerf_args)

os.makedirs(extra_args['output'], exist_ok=True)

z_near, z_far = extra_args['z_near'], extra_args['z_far']
model, pipeline = make_model(
    inerf_args, inerf_args_dict, extra_args, train_dataset, validation_dataset)

def get_image(cam):
    cam = copy.deepcopy(cam)

    rays = gen_rays_only(cam)
    rays = rays.reshape((rays.shape[0]**2, -1))

    rb = model.renderer.render(pipeline, rays, lod_idx=None)
    rgb = rb.rgb.detach().cpu().numpy().reshape((H, W, 3))
    return rgb

def render_pipeline(renderer, pipeline, rays, lod_idx):
    rb = RenderBuffer(xyz=None, hit=None, normal=None, shadow=None, ao=None, dirs=None)
    for ray_pack in rays.split(renderer.render_batch):
        rb  += pipeline.tracer(pipeline.nef, rays=ray_pack, lod_idx=lod_idx, **renderer.kwargs)
    return rb

# curr_pose = torch.tensor(target_poses[0])
# camera = gen_camera(curr_pose, extra_args['focal'], W, H, z_far)


['val', 'train', 'transforms_train.json', 'transforms_test.json', 'transforms_val.json']
<class 'wisp.datasets.formats.rtmv_dataset.RTMVDataset'>
False
<class 'wisp.datasets.formats.nerf_standard_dataset.NeRFSyntheticDataset'>
True


loading data:   0%|          | 0/4 [00:00<?, ?it/s]

loading data: 100%|██████████| 4/4 [00:00<00:00, 48.76it/s]


128.66666666666666


loading data: 100%|██████████| 67/67 [00:01<00:00, 61.87it/s]


155.46740898662756


In [8]:
N = 300
cube_side_len = 2.4
t = np.linspace(-cube_side_len/2, cube_side_len/2, N+1)
batch_size = 4096

query_pts = np.stack(np.meshgrid(t, t, t), -1).astype(np.float32)
print(query_pts.shape)
sh = query_pts.shape
pts = torch.tensor(query_pts.reshape([-1,3]), dtype=torch.float32).to(extra_args['device'])
dirs = torch.zeros_like(pts)
dirs[:,0] = 1

(301, 301, 301, 3)


In [9]:
def batchify(*data, batch_size=1024, device="cpu", progress=True):
  assert all(sample is None or sample.shape[0] == data[0].shape[0] for sample in data), \
    "Sizes of tensors must match for dimension 0."

  def batchifier():
    size, batch_offset = data[0].shape[0], 0
    while batch_offset < size:
      batch_slice = slice(batch_offset, batch_offset + batch_size)
      yield [sample[batch_slice].to(device) if sample is not None else sample for sample in data]
      batch_offset += batch_size

  iterator = batchifier()
  if not progress:
      return iterator
  total = (data[0].shape[0] - 1) // batch_size + 1

  return tqdm(iterator, total=total)

In [10]:
radiance_samples = []
for (batch_pts, batch_dirs,) in batchify(pts, dirs, batch_size=batch_size, device='cuda'):
  with torch.no_grad():
    radiance_batch = pipeline.nef.rgba(batch_pts, batch_dirs)['density'].detach().cpu().numpy()
  radiance_samples.append(radiance_batch)


100%|██████████| 6658/6658 [00:06<00:00, 955.43it/s] 


In [11]:
all_radiances = np.concatenate(radiance_samples).reshape((N+1,N+1,N+1))

In [22]:
all_radiances.mean()

0.6628446

In [12]:
# print(raw.shape)
# plt.hist(np.maximum(0,sigma.ravel()), log=True)
# plt.show()
# print(raw.shape)
# plt.hist(np.maximum(0,sigma.ravel()), log=True)
# plt.show()

### Marching cubes with [PyMCubes](https://github.com/pmneila/PyMCubes)
Change `threshold` to use a different sigma threshold for the isosurface

In [13]:
import mcubes

threshold = 20.
print('fraction occupied', np.mean(all_radiances > threshold))
# all_radiances = all_radiances.transpose((1,0,2))
vertices, triangles = mcubes.marching_cubes(all_radiances, threshold)
print('done', vertices.shape, triangles.shape)

### Uncomment to save out the mesh
# mcubes.export_mesh(vertices, triangles, f"logs/lego_example/lego_{}.dae", "lego")

os.makedirs('logs/examples', exist_ok=True)
vertices = (vertices / N - 0.5) * cube_side_len

fraction occupied 0.011615237795040215
done (185503, 3) (370892, 3)


In [14]:
# print(vertices.min(axis=0))
# print(vertices.mean(axis=0))
# print(vertices.max(axis=0))
# print()


# print(vertices.min(axis=0))
# print(vertices.mean(axis=0))
# print(vertices.max(axis=0))
# print()
vertices = vertices @ np.array([[0,-1,0], [0,0,-1], [1,0,0]]).T
vertices = (vertices - translation_offset.numpy()) / translation_scale.numpy()
pickle.dump((vertices, triangles), open('logs/examples/cheezit.pkl', 'wb'))

In [15]:
# robo_pos = np.array([-0.00032, -0.66887, 0.6 - 7.70155/10])
# T = np.eye(4)
# T[:3,:3] = R.from_euler('xyz', [math.pi/2, math.pi, 0]).as_matrix()
# T[:3,-1] = np.array((0, -0.66887 + 0.6, -0.033828)) - robo_pos
# T

In [16]:
# np.set_printoptions(precision=3, suppress=True)
# obj_act_pose_path = '/home/saptarshi/dev/CustomComposer/workdir/cheezit_5_2_val4_noflip_flip_21_new_2/pose_actual/iter_0.pkl'
# obj_act_pose = pickle.load(open(obj_act_pose_path, 'rb'))
# obj_act_pose

In [17]:
import trimesh
mesh = trimesh.Trimesh(vertices, triangles)

In [21]:
len(vertices)

185503

In [18]:
print(vertices.min(axis=0))
print(vertices.mean(axis=0))
print(vertices.max(axis=0))

[-0.1848787   0.48444498 -0.02845409]
[-0.00188006  0.61040096  0.12652069]
[0.20192113 0.84999999 0.34417392]


In [19]:
cc = trimesh.graph.connected_components(mesh.face_adjacency, min_len=3)
mask = np.zeros(len(mesh.faces), dtype=np.bool)
mask[np.concatenate(cc)] = True
mesh.update_faces(mask)

/tmp/ipykernel_1186919/551277394.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.zeros(len(mesh.faces), dtype=np.bool)


In [20]:
# save mesh as a open3d pointcloud


In [ ]:
# mat = np.eye(4)
# mat[:2,:2] = np.array([[0,1],[1,0]])
# mesh.apply_transform(mat)
mesh.show()

In [16]:
mesh.fill_holes()

True

### Save out video with [pyrender](https://github.com/mmatl/pyrender)

In [74]:
os.environ["PYOPENGL_PLATFORM"] = "egl"
import pyrender

def pose_spherical(theta, phi, radius):
  c2w = trans_t(radius)
  c2w = rot_phi(phi/180.*np.pi) @ c2w
  c2w = rot_theta(theta/180.*np.pi) @ c2w
  c2w = torch.tensor([[-1.,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]]) @ c2w
  return c2w.numpy()

In [76]:
scene = pyrender.Scene()
scene.add(pyrender.Mesh.from_trimesh(mesh, smooth=False))

# Set up the camera -- z-axis away from the scene, x-axis right, y-axis up
camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)

camera_pose = pose_spherical(90., 0., 1.)
# camera_pose = 
nc = pyrender.Node(camera=camera, matrix=camera_pose)
scene.add_node(nc)

# Set up the light -- a point light in the same spot as the camera
light = pyrender.PointLight(color=np.ones(3), intensity=4.0)
nl = pyrender.Node(light=light, matrix=camera_pose)
scene.add_node(nl)

# Render the scene
r = pyrender.OffscreenRenderer(640, 480)
color, depth = r.render(scene)

# plt.imshow(color)
# plt.show()
# plt.imshow(depth)
# plt.show()


In [83]:
imgs = []
for th in np.linspace(0, 360., 120+1)[:-1]:
    camera_pose = pose_spherical(0, th, 1.)
    rot = np.eye(4); rot[:3,:3] = R.from_quat([0, 0, 1/np.sqrt(2), 1/np.sqrt(2)]).as_matrix()
    camera_pose = camera_pose @ rot
    scene.set_pose(nc, pose=camera_pose)
    imgs.append(r.render(scene)[0])
f = 'logs/cheezit/cheezit_mesh.mp4'
os.makedirs(os.path.dirname(f), exist_ok=True)
imageio.mimwrite(f, imgs, fps=30)
print('done')

done


In [84]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open(f,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls autoplay loop>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)